In [6]:
from web3 import Web3
import json
import pandas as pd
import numpy as np

# Connect to Ganache
ganache_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))
assert web3.is_connected(), "Web3 is not connected"

# Contract address and ABI
contract_address = web3.to_checksum_address("0xdd89360a2f53aeed7dad0dea4ac8c974b9dc272a")
contract_abi = [
    {
        "inputs": [{"internalType": "address","name": "device","type": "address"}],
        "name": "authorizeDevice","outputs": [],"stateMutability": "nonpayable","type": "function"
    },
    {
        "inputs": [{"internalType": "address","name": "device","type": "address"}],
        "name": "revokeDevice","outputs": [],"stateMutability": "nonpayable","type": "function"
    },
    {
        "inputs": [
            {"internalType": "string","name": "deviceID","type": "string"},
            {"internalType": "string","name": "dataType","type": "string"},
            {"internalType": "string","name": "value","type": "string"}
        ],
        "name": "storeData","outputs": [],"stateMutability": "nonpayable","type": "function"
    },
    {
        "inputs": [],"stateMutability": "nonpayable","type": "constructor"
    },
    {
        "inputs": [{"internalType": "address","name": "","type": "address"}],
        "name": "authorizedDevices","outputs": [{"internalType": "bool","name": "","type": "bool"}],
        "stateMutability": "view","type": "function"
    },
    {
        "inputs": [
            {"internalType": "string","name": "deviceID","type": "string"},
            {"internalType": "uint256","name": "index","type": "uint256"}
        ],
        "name": "getDataByIndex",
        "outputs": [
            {"internalType": "string","name": "","type": "string"},
            {"internalType": "string","name": "","type": "string"},
            {"internalType": "uint256","name": "","type": "uint256"}
        ],
        "stateMutability": "view","type": "function"
    },
    {
        "inputs": [{"internalType": "string","name": "deviceID","type": "string"}],
        "name": "getDataCount",
        "outputs": [{"internalType": "uint256","name": "","type": "uint256"}],
        "stateMutability": "view","type": "function"
    },
    {
        "inputs": [],"name": "owner",
        "outputs": [{"internalType": "address","name": "","type": "address"}],
        "stateMutability": "view","type": "function"
    }
]

# Load contract
contract = web3.eth.contract(address=contract_address, abi=contract_abi)
print("✅ Connected to contract")

# Define the device ID
device_id = "device001"

# Get total records for the device
total_records = contract.functions.getDataCount(device_id).call()
print(f"Total IoT records stored for {device_id}: {total_records}")

# Fetch all records and structure into a DataFrame
data = []
for i in range(total_records):
    data_type, data_value, timestamp = contract.functions.getDataByIndex(device_id, i).call()
    data.append({
        "timestamp": timestamp,
        "device_id": device_id,
        "data_type": data_type,
        "data_value": data_value
    })

df = pd.DataFrame(data)

# Only process if data is present
if not df.empty:
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
    df["numeric_value"] = df["data_value"].str.extract(r'(\d+\.?\d*)').astype(float)
    df.fillna(0, inplace=True)
    df.to_csv("cleaned_iot_data.csv", index=False)
    print("✅ Cleaned IoT data saved successfully as cleaned_iot_data.csv")
else:
    print("⚠️ No data found for the specified device ID.")


✅ Connected to contract
Total IoT records stored for device001: 0
⚠️ No data found for the specified device ID.
